In [1]:
# Importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image
from keras.preprocessing import image
import cv2
import mediapipe as mp
import csv

In [3]:
current_directory = os.getcwd()

'c:\\Users\\Gokul\\Documents\\Pose_detection_project'

In [12]:
# Image to be detected
path_of_image = os.path.join(current_directory, 'DATASET/TEST/warrior2/00000003.jpg')
# path_of_image = "C:/Users/Gokul/Desktop/Project/DATASET/TEST/warrior2/00000003.jpg"

'c:\\Users\\Gokul\\Documents\\Pose_detection_project\\DATASET\\TEST\\warrior2\x0000003.jpg'

In [21]:
#Importing trained models
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import pickle

model_path_cnn = os.path.join(current_directory, 'network.h5')
model_path_rf = os.path.join(current_directory, 'body_language.pkl')

model = load_model(model_path_cnn) # CNN model

# with open('C:/Users/Gokul/Desktop/Project/body_language.pkl', 'rb') as f:
#     model2 = pickle.load(f) # Randomforest model

with open(model_path_rf, 'rb') as f:
    model2 = pickle.load(f)


In [16]:
# Function to process an image and extract landmarks
def process_image(image_path,holistic):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to read image: {image_path}")
        return

    # Convert image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Make Detections
    results = holistic.process(image_rgb)

    num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
    landmarks = []
    for val in range(1, num_coords+1):
        landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

    full_path_of_csv = os.path.join(current_directory, "cordi.csv")

    with open(full_path_of_csv, mode='w', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)
    f.close()

    # points = pd.DataFrame(columns=landmarks)

    try:
        # Extract Pose landmarks
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
        
        # Extract Face landmarks
        face = results.face_landmarks.landmark
        face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
        
        # Concate rows
        row = pose_row + face_row
        # return row

        # points = points._append(row,ignore_index = False)
        
        # Append class name 
        # row.insert(0, class_name)

        # # Export to CSV
        with open(full_path_of_csv, mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(row) 
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
    # return points

# Release resources
    holistic.close()

In [17]:
# Initialize holistic model
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
holistics = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
process_image(path_of_image,holistics)

In [19]:
full_path_of_csv = os.path.join(current_directory, "cordi.csv")
df = pd.read_csv(full_path_of_csv) # coordinates obtained
df

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,0.461584,0.128652,-0.196279,0.99978,0.477099,0.110123,-0.189061,0.99846,0.481574,0.111407,...,-0.00188,0.0,0.473865,0.102954,-0.011354,0.0,0.475884,0.100551,-0.012321,0.0


In [26]:
# CNN model Output
from keras.preprocessing import image
img2 = image.load_img(path_of_image, target_size=(224, 224))
x = image.img_to_array(img2)
x = np.expand_dims(x, axis=0)
img_data = preprocess_input(x)
classes = model.predict(img_data)
classname = ["downdog","Goddess","plank","tree","warrior"]

predicted_classes = [classname[classes.argmax()] for prediction in classes]
predicted_classes
# print(classname)
# print(classes) 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step


['warrior']

In [27]:
# Randomforest Classifier Output
pred2 = model2.predict(df)
pred2

array(['warrior'], dtype=object)

In [28]:
# Display output on a window
# Load the image
final = cv2.imread(path_of_image)

# Create a named window
cv2.namedWindow("Output", cv2.WINDOW_NORMAL)
# Set the window size (width, height)
window_size = (800, 600)
cv2.resizeWindow("Output", *window_size)

f2 = cv2.resize(final,window_size)

cv2.putText(f2,f"The prediction using CNN is {classes}.",(10,20),cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)
cv2.putText(f2,f"The prediction using RandomForest classifier is {pred2[0]}.",(10,40),cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)
cv2.imshow("Output",f2)
# Wait for a key press
cv2.waitKey(0)

# Close the window
cv2.destroyAllWindows()